In [54]:
# mlp for regression
from numpy import sqrt
from pandas import read_csv
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [55]:
# load the dataset
df = read_csv("Memory_SolarTrainingData.csv", index_col=0)

In [56]:
df.head()

,temp,feelslike,dew,humidity,precip,snow,snowdepth,windgust,windspeed,winddir,...,cloudcover,visibility,solarradiation,solarenergy,uvindex,year,month,day,time,Percent Output
197,25.0,14.1,11.4,55.86,0.0,0,0,18.9,11.7,10.0,...,97.5,9.9,18.0,0.1,0.0,2018,1,1,8.0,0.014921
198,25.0,14.1,11.4,55.86,0.0,0,0,18.9,11.7,10.0,...,97.5,9.9,18.0,0.1,0.0,2018,1,1,8.0,0.015856
199,25.0,14.1,11.4,55.86,0.0,0,0,18.9,11.7,10.0,...,97.5,9.9,18.0,0.1,0.0,2018,1,1,8.0,0.003378
201,25.0,14.1,11.4,55.86,0.0,0,0,18.9,11.7,10.0,...,97.5,9.9,18.0,0.1,0.0,2018,1,1,8.0,0.017996
202,25.0,14.1,11.4,55.86,0.0,0,0,18.9,11.7,10.0,...,97.5,9.9,18.0,0.1,0.0,2018,1,1,8.0,0.016960


In [57]:
# split into past, current, and future data
pastWeatherAndSupply = df.iloc[:-2]

currentWeather = df.iloc[-2]
currentWeather = currentWeather.drop(['Percent Output']) #dropping supply column for testing
currentWeather = currentWeather.values[:].reshape((1,20))

nextHourWeather = df.iloc[-1]
nextHourWeather = nextHourWeather.drop(['Percent Output']) #dropping supply column for testing
nextHourWeather = nextHourWeather.values[:].reshape((1,20))

In [58]:
# split into input and output columns
X, y = pastWeatherAndSupply.values[:, :-1], pastWeatherAndSupply.values[:, -1]

In [59]:
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6183, 20) (3046, 20) (6183,) (3046,)


In [60]:
# determine the number of input features
n_features = X_train.shape[1]
print(n_features)

20


In [61]:
# define model
model = Sequential()
model.add(Dense(10, activation='sigmoid', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='he_normal'))

In [62]:
# compile the model
model.compile(optimizer='adam', loss='mse')

In [63]:
# fit the model
model.fit(X_train, y_train, epochs=300, batch_size=10, verbose=0)

In [64]:
# evaluate the model
error = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f' % (error, sqrt(error)))

MSE: 0.027, RMSE: 0.163


In [65]:
# Making a prediction for integration with control code
# IMPROVEMENT AREA: Sometimes the model will predict a number greater than 1. This should not happen.
current_prediction = model.predict(currentWeather)
if (current_prediction < 0):
    current_prediction = 0
print('Predicted: %.3f' % current_prediction)

future_prediction = model.predict(nextHourWeather)
if (current_prediction < 0):
    current_prediction = 0
print('Predicted: %.3f' % future_prediction)

1/1 [==============================] - 0s 61ms/step
Predicted: 0.493
1/1 [==============================] - 0s 23ms/step
Predicted: 0.493


In [66]:
# Return predictions in a list
#return([current_prediction, future_prediction])